In this notebook, I want to compute the uncertainty of the equilibrium lattice constant and the cohesive energy from the bootstrap ensembles.

In [1]:
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
import sys

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use("default")

WORK_DIR = Path().absolute()
RES_DIR = WORK_DIR / "results" / "dropout"
sys.path.append(str(WORK_DIR.parent))

In [2]:
from energyvslatconst.relaxation_latconst import equilibrate_graphite

In [3]:
# Read setting file
ROOT_DIR = WORK_DIR.parent
with open(ROOT_DIR / "settings.json", "r") as f:
    settings = json.load(f)
partition = settings["partition"]
suffix = "_".join([str(n) for n in settings["Nnodes"]])
RES_DIR = WORK_DIR / "results" / "dropout" / f"{partition}_partition_{suffix}"

In [4]:
a0_c0_e0_file = RES_DIR / "uncertainty_latconst_ecoh_graphite.npz"
if a0_c0_e0_file.exists():
    a0_c0_e0 = np.load(a0_c0_e0_file)
    a0_list = a0_c0_e0["a0_list"]
    c0_list = a0_c0_e0["c0_list"]
    e0_list = a0_c0_e0["e0_list"]
else:
    a0_list = []
    c0_list = []
    e0_list = []

    ainit = 2.466  # From materials project for graphite-like structure
    cinit = 3.348  # From https://doi.org/10.1073/pnas.2134173100
    for ii in tqdm(range(100)):
        # Equilibration
        a0, c0, e0 = equilibrate_graphite(
            "DUNN_best_train", ainit, cinit, active_member_id=ii + 1
        )
        a0_list = np.append(a0_list, a0)
        c0_list = np.append(c0_list, c0)
        e0_list = np.append(e0_list, e0)

    np.savez(a0_c0_e0_file, a0_list=a0_list, c0_list=c0_list, e0_list=e0_list)

In [5]:
# Lattice constant a
print("In-plane lattice constant a")
print("Mean:", np.mean(a0_list))
print("Stdev:", np.std(a0_list))

In-plane lattice constant a
Mean: 2.46424759875
Stdev: 0.005803284461697692


In [6]:
# Lattice constant c
print("Out-of-plane lattice constant c")
print("Mean:", np.mean(c0_list))
print("Stdev:", np.std(c0_list))

Out-of-plane lattice constant c
Mean: 3.3456208274999995
Stdev: 0.007878911750918035


In [7]:
# Cobesive energy
print("Cohesive energy")
print("Mean:", np.mean(e0_list))
print("Stdev:", np.std(e0_list))

Cohesive energy
Mean: -8.100626455748051
Stdev: 0.01552393586934175
